In [25]:
import pandas as pd
import folium
import numpy as np
import os
import googlemaps
import requests
from textwrap import shorten
from bs4 import BeautifulSoup
import re

In [43]:
df = pd.read_csv('FortLauderdaleCoastalBrowardCondoMap - BrowardCondoPipelineMap.csv',dtype='str')

## Data clean

In [44]:
df = df.dropna(subset='Address')

In [45]:
df = df.drop(columns=['Developer Checked','Emailed PR'])

In [46]:
df['Full_Address'] = df['Address'] + ' ' + df['City'] + ' , FL'

## Geocode

In [30]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [47]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'geo_address' column and store the results in 'geocoded' column
df['geocoded'] = df['Full_Address'].apply(geocode)

## Link Formatter

In [50]:
import pandas as pd
import textwrap

def shorten_text(text, width=20, placeholder="..."):
    """
    Shorten the text to a specified width without breaking words.

    Parameters:
    - text (str): The original text.
    - width (int): The maximum width of the shortened text.
    - placeholder (str): The string to append if the text is shortened.

    Returns:
    - str: The shortened text.
    """
    return textwrap.shorten(text, width=width, placeholder=placeholder)

def get_remaining_desc(description, short_desc):
    """
    Get the remaining part of the description after the short description.

    Parameters:
    - description (str): The full description.
    - short_desc (str): The shortened description.

    Returns:
    - str: The remaining description.
    """
    if description.startswith(short_desc):
        return description[len(short_desc):]
    else:
        # If 'short_description' is not at the start, handle accordingly
        return description

# Apply the shortening function to create 'short_description'
df["short_description"] = df["Description"].apply(lambda s: shorten_text(s, width=20, placeholder="..."))

# Create 'remaining_desc' by removing the 'short_description' from 'Description'
df['remaining_desc'] = df.apply(lambda row: get_remaining_desc(row['Description'], row['short_description']), axis=1)

# Ensure that 'URL' does not have missing values
df['URL'] = df['URL'].fillna('#')  # Using '#' as a fallback URL

# Create the 'Story Link' column with HTML anchor tags
df["Story Link"] = (
    '<a href="' + df["URL"] + '" target="_blank" rel="noopener noreferrer">' +
    df["short_description"] +
    "</a>" +
    df["remaining_desc"].fillna('')
)

# Display the DataFrame to verify
print(df[['Description', 'short_description', 'remaining_desc', 'Story Link']].head())


                            Description   short_description  \
0                    43-story, 775-unit  43-story, 775-unit   
1                     20-story, 36-unit   20-story, 36-unit   
2                    28-story, 44-unit    28-story, 44-unit   
3  two 23-story condos, 160 units total     two 23-story...   
4                    40-story, 384-unit  40-story, 384-unit   

                         remaining_desc  \
0                                         
1                                         
2                                         
3  two 23-story condos, 160 units total   
4                                         

                                          Story Link  
0  <a href="https://therealdeal.com/miami/2024/10...  
1  <a href="https://therealdeal.com/miami/2024/10...  
2  <a href="https://therealdeal.com/miami/2024/10...  
3  <a href="https://therealdeal.com/miami/2024/10...  
4  <a href="https://therealdeal.com/miami/2024/11...  


In [52]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x == 'None' else x.strip('()').split(', ', 1)).apply(pd.Series)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

In [53]:
df.columns

Index(['Address', 'Project name', 'Developer', 'Description', 'City', 'URL',
       'Full_Address', 'geocoded', 'short_description', 'remaining_desc',
       'Story Link', 'lat', 'lon'],
      dtype='object')

In [54]:
df['Story Link'].iloc[0]

'<a href="https://therealdeal.com/miami/2024/10/30/dependable-equities-starts-sales-of-fort-lauderdale-condos/" target="_blank" rel="noopener noreferrer">43-story, 775-unit</a>'

In [55]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd
import textwrap

def popup_html(row):
    """
    Generate HTML content for the popup with relevant information.
    
    Parameters:
    - row (pd.Series): A row from the DataFrame.
    
    Returns:
    - str: HTML string for the popup content.
    """
    # Retrieve the necessary fields, handling missing values
    address = str(row.get('Address', 'N/A')).strip()
    project_name = str(row.get('Project name', 'N/A')).strip()
    developer = str(row.get('Developer', 'N/A')).strip()
    description = str(row.get('Description', 'N/A')).strip()
    story_link = str(row.get('Story Link', '#')).strip()
    
    # Optionally, wrap text for better display
    wrapped_address = '<br>'.join(textwrap.wrap(address, width=40, break_long_words=False))
    wrapped_project = '<br>'.join(textwrap.wrap(project_name, width=40, break_long_words=False))
    wrapped_developer = '<br>'.join(textwrap.wrap(developer, width=40, break_long_words=False))
    wrapped_description = '<br>'.join(textwrap.wrap(description, width=60, break_long_words=False))
    
    # Construct the HTML content
    html = f"""
    <div class="popup-content">
        <strong>Project Name:</strong> {wrapped_project}<br>
        <strong>Developer:</strong> {wrapped_developer}<br>
        <strong>Address:</strong> {wrapped_address}<br>
        <strong>Description:</strong> {wrapped_description}<br>
        <a href="{story_link}" target="_blank">Read More</a>
    </div>
    """
    return html

# Initialize the map centered around the mean latitude and longitude
map_center = [df['lat'].mean(), df['lon'].mean()]
m = folium.Map(location=map_center, zoom_start=10)

# Add the custom Mapbox tile layer
folium.TileLayer(
    tiles='https://api.mapbox.com/styles/v1/mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoidHJkZGF0YSIsImEiOiJjamc2bTc2YmUxY2F3MnZxZGh2amR2MTY5In0.QlOWqB-yQNrNlXD0KQ9IvQ',
    attr='Mapbox',
    name='Streets',
    overlay=True,
    control=False,
    show=False,
    min_zoom=1,
    max_zoom=20
).add_to(m)

# Add custom CSS to style the popups
custom_css = """
<style>
    .popup-content {
        max-width: 300px; /* Adjusts the maximum width of the popup */
        font-size: 12px;   /* Adjusts the font size */
        word-wrap: break-word; /* Ensures long words wrap to the next line */
    }
    .leaflet-popup-content-wrapper {
        background-color: white;
        border: 1px solid gray;
        border-radius: 3px;
        padding: 5px;
    }
</style>
"""

# Add the custom CSS to the map
m.get_root().html.add_child(folium.Element(custom_css))

# Create and add the title to the map
title_html = '''
             <h3 align="center" style="font-size:16px"><b>Known Alexander Properties</b></h3>
             '''
m.get_root().html.add_child(folium.Element(title_html))

# Initialize a MarkerCluster to handle multiple markers efficiently
marker_cluster = MarkerCluster().add_to(m)

# Loop through the DataFrame to create markers
for i, row in df.iterrows():
    # Create popup HTML content
    popup_content = popup_html(row)
    
    # Create a popup object with an IFrame to control dimensions
    popup = folium.Popup(folium.IFrame(html=popup_content, width=300, height=200), max_width=300)
    
    # Create a CircleMarker with the popup
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        popup=popup,
        color='red',
        fill=True,
        fill_color='red',
        radius=7,
        weight=1,          # Thickness of the circle border
        opacity=0.8,       # Opacity of the circle border
        fill_opacity=0.6   # Opacity of the fill
    ).add_to(marker_cluster)

# Add layer control to toggle different layers
folium.LayerControl().add_to(m)

# Display the map
m


In [41]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd
import textwrap

def tooltip_html(row_index, max_chars=30):
    """
    Generate HTML content for the tooltip with line breaks at word boundaries
    and without any labels.
    
    Parameters:
    - row_index (int): The index of the row in the DataFrame.
    - max_chars (int): Maximum number of characters per line.
    
    Returns:
    - str: HTML string for the tooltip content.
    """
    # Retrieve the address from 'Display Properties'
    address = df.at[row_index, 'Display Properties']
    address_str = str(address).strip() if pd.notnull(address) else 'N/A'
    
    # Use textwrap to wrap text without breaking words
    wrapped_address = '<br>'.join(textwrap.wrap(address_str, width=max_chars, break_long_words=False))
    
    # Construct the tooltip HTML without any labels
    tooltip_content = f'<div class="popup-content">{wrapped_address}</div>'
    return tooltip_content

# Initialize the map centered around the mean latitude and longitude
map_center = [df['lat'].mean(), df['lon'].mean()]
m = folium.Map(location=map_center, zoom_start=5)

# Add the custom Mapbox tile layer
folium.TileLayer(
    tiles = 'https://api.mapbox.com/styles/v1/mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoidHJkZGF0YSIsImEiOiJjamc2bTc2YmUxY2F3MnZxZGh2amR2MTY5In0.QlOWqB-yQNrNlXD0KQ9IvQ',
    attr='Mapbox',
    name='Streets',
    overlay=True,
    control=False,
    show=False,
    min_zoom=1,
    max_zoom=20
).add_to(m)

# Add custom CSS to style the tooltips
custom_css = """
<style>
    .popup-content {
        max-width: 200px; /* Adjusts the maximum width of the tooltip */
        font-size: 12px;   /* Adjusts the font size */
        word-wrap: break-word; /* Ensures long words wrap to the next line */
    }
    .leaflet-tooltip {
        background-color: white;
        border: 1px solid gray;
        border-radius: 3px;
        padding: 5px;
    }
</style>
"""

# Add the custom CSS to the map
m.get_root().html.add_child(folium.Element(custom_css))

# Create and add the title to the map
title_html = '''
             <h3 align="center" style="font-size:16px"><b>Known Alexander Properties</b></h3>
             '''
m.get_root().html.add_child(folium.Element(title_html))

# Initialize a MarkerCluster to handle multiple markers efficiently
marker_cluster = MarkerCluster().add_to(m)

# Loop through the DataFrame to create markers
for i, row in df.iterrows():
    # Create tooltip HTML content
    tooltip_content = tooltip_html(i)
    
    # Create a CircleMarker without the 'icon' parameter
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        tooltip=folium.Tooltip(tooltip_content, sticky=True),
        color='red',
        fill=True,
        fill_color='red',
        radius=7,
        weight=1,          # Thickness of the circle border
        opacity=0.8,       # Opacity of the circle border
        fill_opacity=0.6   # Opacity of the fill
    ).add_to(marker_cluster)

# Add layer control to toggle different layers
folium.LayerControl().add_to(m)

# Display the map
m

In [42]:
m.save('index.html')

In [43]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Alexander_Map
